## 0. Environment Setup
We create a smiliar environemnt that simulate the inputs and variables as per in ciop jobs

In [86]:
model_config_file='https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=70EE8A2A6898282CE352F7F635C4B34ED5FCEB13' #Reference to fanfar-forecast.zip
TMPDIR="/workspace/tmpdir2"
TMPDIR

[1] "/workspace/tmpdir2"

In [87]:
app.date = format(Sys.time(), "%Y%m%d_%H%M")

## set application name
app.name = "forecast"
## ------------------------------------------------------------------------------
## flag which environment is used, if not set
if(!exists("app.sys")){
    app.sys ="tep"
}
## ------------------------------------------------------------------------------
## load rciop package and set working directory to TMPDIR when running on TEP 
if(app.sys=="tep"){
    library("rciop")

    rciop.log ("DEBUG", " *** hypeapps-forecast *** TEP hydrological modelling applications ***", "/node_forecast/run.R")
    rciop.log ("DEBUG", " rciop library loaded", "/node_forecast/run.R")

    if (! dir.exists(TMPDIR)) {
        dir.create(TMPDIR,recursive=TRUE,mode="0775")
    }
    
    curDir <- getwd()
    print(curDir)
    setwd(TMPDIR)
    rciop.log("DEBUG", paste(" R session working directory set to ",TMPDIR,sep=""), "/node_forecast/run.R")
}

NULL


Some rciop functions may need to be overridden in the notebook environment.
Set to your current needs.

In [88]:
ENABLE_STUB_RCIOP_LOG <- FALSE
#ENABLE_STUB_RCIOP_LOG <- TRUE

if (ENABLE_STUB_RCIOP_LOG == TRUE) {
    message("rciop.log() overridden by local stub function...")
    
    rciop.log <- function(type, message, proc='') {

        type <- type
        msg <- message
    if (proc=='') { proc <- "user process" }

    out.msg <- paste(format(Sys.time(), format="%Y-%m-%dT%H:%M:%S"), "[", type, "][", proc, "] ", msg, sep="")

    message(out.msg)
    } # rciop.log

} # end stub

In [89]:
#ENABLE_STUB_RCIOP_GETPARAM <- FALSE
ENABLE_STUB_RCIOP_GETPARAM <- TRUE

if (ENABLE_STUB_RCIOP_GETPARAM == TRUE) {
    message("rciop.getparam() overridden by local stub function...")
    # Change according to your test objective.
    # Would normally return values defined in application.xml for specified ids'.
    
    rciop.getparam <- function(id) {
        value <- NULL # or ""
        
        if (id == "model") {
            value <- "https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=230286758E25FD953AF736DE85F01CDF9C539EC7"
        }
        else if (id == "idate" ) {
            value <- "2018-09-01"
        }
        else if (id == "hydmodel" ) {
            c()
            #value[1] <- "Niger-HYPE"
            #value[2] <- "WestAfrica-HYPE"
            value <- c('Niger-HYPE','WestAfrica-HYPE')
        }
        else {
            message("rciop.get_param:id not found...")
        }
        
        return (value)
    } # rciop.getparam

} # end stub

rciop.getparam() overridden by local stub function...


Add your code to test...
Remove current, or paste other code or add additional code blocks.

In [90]:
## Search for the main data configuration and download the object

# print
rciop.log("INFO", paste("Processing input:", model_config_file, sep=" "))

# Query the input reference
message(paste("opensearch-client '",model_config_file,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",model_config_file,"' enclosure")
res_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", res_enclosure)

# Download the dir(s) or file(s)
# File in this case
main_config_file <- rciop.copy(res_enclosure,target=TMPDIR,uncompress=TRUE)
#dir.create(paste(TMPDIR,"fanfar-forecast",sep="/"),recursive=TRUE,mode="0775")
#main_config_file <- rciop.copy(res_enclosure,target=paste(TMPDIR,"fanfar-forecast",sep="/"),uncompress=TRUE)

if (main_config_file$exit.code==0) {
    local.main_config_file <- main_config_file$output
}
local.main_config_file

opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=70EE8A2A6898282CE352F7F635C4B34ED5FCEB13' enclosure


[1] " -o /workspace/tmpdir2 https://store.terradue.com/hydro-smhi/models/fanfar-forecast.zip"


[1] "/workspace/tmpdir2/fanfar-forecast"

In [92]:
# Parse the main configuration file

#local.main_config_dir
file <- "v1/dependencies.txt"
main_config_file <- paste(local.main_config_file,file,sep="/")
#main_config_file <- local.main_config_file

if (! file.exists(main_config_file)) {
    q(save="no",status=1)
}

#main_config_data <- read.csv2(main_config_file,header=FALSE,sep=";")
main_config_data <- read.csv2(main_config_file,header=TRUE,sep=";")
main_config_data

for (r in 1:nrow(main_config_data)) {
    subdir <- main_config_data[r,'localdirectory']
    if (subdir == 'hype-model') {
        modelDataConfigSubDir  <- main_config_data[r,'localdirectory'] # Intended to be used as dir name for rciop.copy TMPDIR/subdir/
        modelDataConfigUrl     <- main_config_data[r,'url']
        modelDataConfigQuery   <- main_config_data[r,'searchquery']
        modelDataConfigComment <- main_config_data[r,'comment']
    }
    if (subdir == 'hydrogfd-config') {
        hydroGFDDataConfigSubDir  <- main_config_data[r,'localdirectory'] # Intended to be used as dir name for rciop.copy TMPDIR/subdir/
        hydroGFDDataConfigUrl     <- main_config_data[r,'url']
        hydroGFDDataConfigQuery   <- main_config_data[r,'searchquery']
        hydroGFDDataConfigComment <- main_config_data[r,'comment']
    }
}
#modelDataConfigSubDir
#modelDataConfigUrl
#modelDataConfigQuery
#modelDataConfigComment
#hydroGFDDataConfigSubDir
#hydroGFDDataConfigUrl
#hydroGFDDataConfigQuery
#hydroGFDDataConfigComment

localdirectory,url,searchquery,comment
hype-model,https://recast.terradue.com/t2api/search/hydro-smhi/models,?uid=E21AB563CFA40CC36D818D67DACBB63B35790A16,# zip-file: westafrica-hype-model/v1.3.6
hydrogfd-config,https://recast.terradue.com/t2api/search/hydro-smhi/models,?uid=BC9DF45931AB9F68196F465D29AD74CE3F4C95F3,# zip-file: hydrogfd.zip/v2 (old:hydrogfd-data)


In [93]:
## Search for the model data configuration and download the data

url   <- modelDataConfigUrl
query <- modelDataConfigQuery

# Query the model data config reference
message(paste("opensearch-client '",url,query,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",url,query,"' enclosure")
res_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", res_enclosure)

# Download the dir(s) or file(s)
# Dir in this case
model_data_dir <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE)

if (model_data_dir$exit.code==0) {
    local.model_data_dir <- model_data_dir$output
}
local.model_data_dir

# Either keep a unified directory structure that are named the same between different versions, or
# output variables for dirs/files etc. that are named differently.

opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=E21AB563CFA40CC36D818D67DACBB63B35790A16' enclosure


[1] " -o /workspace/tmpdir2 https://store.terradue.com/hydro-smhi/models/westafrica-hype-model.zip"


[1] "/workspace/tmpdir2/westafrica-hype-model"

In [94]:
## Search for the hydrogfd data configuration and download the object

url   <- hydroGFDDataConfigUrl
query <- hydroGFDDataConfigQuery

# Query the hydrogfd data config reference
message(paste("opensearch-client '",url,query,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",url,query,"' enclosure")
res_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", res_enclosure)

# Download the dir(s) or file(s)
# File in this case
hydrogfd_config_file <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE)
#hydrogfd_data_dir <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE,createOutputDirectory=TRUE)
#hydrogfd_data_dir <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE,target=hydroGFDDataConfigSubDir)

if (hydrogfd_config_file$exit.code==0) {
    local.hydrogfd_config_file <- hydrogfd_config_file$output
}
local.hydrogfd_config_file

opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/models?uid=BC9DF45931AB9F68196F465D29AD74CE3F4C95F3' enclosure


[1] " -o /workspace/tmpdir2 https://store.terradue.com/hydro-smhi/models/hydrogfd.zip"


[1] "/workspace/tmpdir2/hydrogfd"

In [95]:
 # Parse the hydrogfd configuration file

file <- "v2/dependencies.txt"
hydrogfd_config_file <- paste(local.hydrogfd_config_file,file,sep="/")
#main_config_file <- local.main_config_file

print(hydrogfd_config_file)

if (! file.exists(hydrogfd_config_file)) {
    q(save="no",status=1)
}

hydrogfd_config_data <- read.csv2(hydrogfd_config_file, header=TRUE, sep=";")
hydrogfd_config_data
#names(model_config_data) <- c('subdir','url','querypattern','info')
for (r in 1:nrow(hydrogfd_config_data)) {
    subdir <- hydrogfd_config_data[r,'localdirectory']
    if (subdir == 'hydrogfdei') {
      gfdHydrogfdeiSubDir  <- hydrogfd_config_data[r,'localdirectory'] # Intended to be used both for dir name (rciop.copy) and part of filename
      gfdHydrogfdeiUrl     <- hydrogfd_config_data[r,'url']
      gfdHydrogfdeiQuery   <- hydrogfd_config_data[r,'searchquery']
      gfdHydrogfdeiComment <- hydrogfd_config_data[r,'comment']
    }
    if (subdir == 'hydrogfdod') {
      gfdHydrogfdodSubDir  <- hydrogfd_config_data[r,'localdirectory']
      gfdHydrogfdodUrl     <- hydrogfd_config_data[r,'url']
      gfdHydrogfdodQuery   <- hydrogfd_config_data[r,'searchquery']
      gfdHydrogfdodComment <- hydrogfd_config_data[r,'comment']
    }
    if (subdir == 'od-daily') {
      gfdOdDailySubDir  <- hydrogfd_config_data[r,'localdirectory']
      gfdOdDailyUrl     <- hydrogfd_config_data[r,'url']
      gfdOdDailyQuery   <- hydrogfd_config_data[r,'searchquery']
      gfdOdDailyComment <- hydrogfd_config_data[r,'comment']
    }
    if (subdir == 'ecoper') {
      gfdEcoperSubDir  <- hydrogfd_config_data[r,'localdirectory']
      gfdEcoperUrl     <- hydrogfd_config_data[r,'url']
      gfdEcoperQuery   <- hydrogfd_config_data[r,'searchquery']
      gfdEcoperComment <- hydrogfd_config_data[r,'comment']
    }
    # Skip subdir 'ei-monthly' and 'od-monthly'
    if (subdir == 'elevation') {
      gfdElevationSubDir  <- hydrogfd_config_data[r,'localdirectory']
      gfdElevationUrl     <- hydrogfd_config_data[r,'url']
      gfdElevationQuery   <- hydrogfd_config_data[r,'searchquery']
      gfdElevationComment <- hydrogfd_config_data[r,'comment']
    }
    if (subdir == 'grid-meta') {
      gfdGridSubDir  <- hydrogfd_config_data[r,'localdirectory']
      gfdGridUrl     <- hydrogfd_config_data[r,'url']
      gfdGridQuery   <- hydrogfd_config_data[r,'searchquery']
      gfdGridComment <- hydrogfd_config_data[r,'comment']
    }
  }
#gfdHydrogfdeiSubDir
#gfdHydrogfdeiUrl
#gfdHydrogfdeiQuery
#gfdHydrogfdeiComment
#gfdHydrogfdodSubDir
#gfdHydrogfdodUrl
#gfdHydrogfdodQuery
#gfdHydrogfdodComment
#gfdOdDailySubDir
#gfdEcoperUrl
#gfdEcoperQuery
#gfdEcoperComment
#gfdElevationSubDir
#gfdElevationUrl
#gfdElevationQuery
#gfdElevationComment
#gfdGridSubDir
#gfdGridUrl
#gfdGridQuery
#gfdGridComment

[1] "/workspace/tmpdir2/hydrogfd/v2/dependencies.txt"


localdirectory,url,searchquery,comment
hydrogfdei,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdei,SMHI,operational]","# HydroGFD2.0,netcdf,yyyymm"
hydrogfdod,https://catalog.terradue.com/hydro-smhi/description,"[hydrogfdod,SMHI,operational]","# HydroGFD2.0,netcdf,yyyymm"
od-daily,https://catalog.terradue.com/hydro-smhi/description,"[od,daily,SMHI,operational]","# HydroGFD2.0,netcdf,yyyymmdd"
ecoper,https://catalog.terradue.com/hydro-smhi/description,"[ecoper,SMHI,operational]","# HydroGFD2.0,netcdf,yyyymmdd00"
elevation,https://recast.terradue.com/t2api/search/hydro-smhi/fanfar/Operational/elevation,?uid=E7BBF23D33590BA045CA51A5095CF1E8D21B7A2D,"# HydroGFD2.0,netcdf"
grid-meta,https://recast.terradue.com/t2api/search/hydro-smhi/fanfar/Operational/grid,?uid=168FFE17C23569EE6286422230674906AE049FF4,"# shape files (point,polygon)"
ei-monthly,https://catalog.terradue.com/hydro-smhi/description,"[ei,monthly,SMHI,operational]","# HydroGFD2.0,netcdf,yyyymm,AGRHYMET"
od-monthly,https://catalog.terradue.com/hydro-smhi/description,"[od,monthly,SMHI,operational]","# HydroGFD2.0,netcdf,yyyymm,AGRHYMET"


[1] grid-meta
8 Levels: ecoper ei-monthly elevation grid-meta hydrogfdei ... od-monthly

[1] https://recast.terradue.com/t2api/search/hydro-smhi/fanfar/Operational/grid
3 Levels: https://catalog.terradue.com/hydro-smhi/description ...

[1] ?uid=168FFE17C23569EE6286422230674906AE049FF4
8 Levels: ?uid=168FFE17C23569EE6286422230674906AE049FF4 ...

[1] # shape files (point,polygon)
6 Levels: # HydroGFD2.0,netcdf ... # shape files (point,polygon)

In [ ]:
## Search for the hydrogfd configuration items and download the data


In [96]:
#opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/fanfar/Operational/grid?uid=168FFE17C23569EE6286422230674906AE049FF4' enclosure
#'https://store.terradue.com/hydro-smhi/fanfar/Operational/grid/grid.meta.zip'

url <- 'https://recast.terradue.com/t2api/search/hydro-smhi/fanfar/Operational/grid'
query <- '?uid=168FFE17C23569EE6286422230674906AE049FF4'

message(paste("opensearch-client '",url,query,"' enclosure", sep=""))
opensearchCmd=paste("opensearch-client '",url,query,"' enclosure")
res_enclosure <- system(command = opensearchCmd,intern = T)
rciop.log("INFO", res_enclosure)

# Download the dir(s) or file(s)
# File in this case
grid_config_file <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE)
#hydrogfd_data_dir <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE,createOutputDirectory=TRUE)
#hydrogfd_data_dir <- rciop.copy(res_enclosure,TMPDIR,uncompress=TRUE,target=hydroGFDDataConfigSubDir)

if (grid_config_file$exit.code==0) {
    local.grid_config_file <- grid_config_file$output
}
local.grid_config_file

opensearch-client 'https://recast.terradue.com/t2api/search/hydro-smhi/fanfar/Operational/grid?uid=168FFE17C23569EE6286422230674906AE049FF4' enclosure


[1] " -o /workspace/tmpdir2 https://store.terradue.com/hydro-smhi/fanfar/Operational/grid/grid.meta.zip"


[1] "/workspace/tmpdir2/grid.meta"